In [1]:
from github import Github
import requests

In [2]:
token = "YOUR_TOKEN_HERE"
g = Github(token)


In [3]:
def get_commit(repo,id):
    try :
        return repo.get_commit(id)
    except:
        return None



In [4]:
def extract_data(raw_data, key):
    try :
        if key in raw_data:
            return raw_data[key]
        else:
            return None
    except:
        return None

In [5]:
def get_file_content(file_url):
    try:
        response = requests.get(file_url)
        if response.status_code == 200:  
            content = response.content.decode('utf-8')
            return content
        else:
            return None
    except:
        return None

In [6]:
repo_name = "apache/openmeetings"
repo = g.get_repo(repo_name)
pulls = repo.get_pulls(state="open")
#repo_name = "apache/kafka"
#repo = g.get_repo(repo_name)
#temp = repo.get_pull(8866)
#pulls = [temp]
review_ids = []
review_datas = []
for pull in pulls:
    reviews = pull.get_reviews()
    for review in reviews:
        print(review.raw_data)
        review_data = {
            "project": repo_name,
            "pull_number": pull.number,
            "review_id": review.raw_data["id"],
            "user": review.raw_data["user"]["login"],
            "user_id": review.raw_data["user"]["id"],
            "message": review.raw_data["body"],
            "state": review.raw_data["state"],
            "submitted_at": review.raw_data["submitted_at"],
            "pull_request_id": review.raw_data["pull_request_url"].split("/")[-1],
            "pull_request_url": review.raw_data["pull_request_url"],
            "commit_id" : review.raw_data["commit_id"] if "commit_id" in review.raw_data else None,  
            "comments" : []
        }
        review_datas.append(review_data)
    
        


{'id': 542920858, 'node_id': 'MDE3OlB1bGxSZXF1ZXN0UmV2aWV3NTQyOTIwODU4', 'user': {'login': 'solomax', 'id': 3870591, 'node_id': 'MDQ6VXNlcjM4NzA1OTE=', 'avatar_url': 'https://avatars.githubusercontent.com/u/3870591?u=caddc61dea58ed1bb424100568fb057cadb63669&v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/solomax', 'html_url': 'https://github.com/solomax', 'followers_url': 'https://api.github.com/users/solomax/followers', 'following_url': 'https://api.github.com/users/solomax/following{/other_user}', 'gists_url': 'https://api.github.com/users/solomax/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/solomax/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/solomax/subscriptions', 'organizations_url': 'https://api.github.com/users/solomax/orgs', 'repos_url': 'https://api.github.com/users/solomax/repos', 'events_url': 'https://api.github.com/users/solomax/events{/privacy}', 'received_events_url': 'https://api.github.com/users/solomax/re

In [7]:
def find_file(files, file_path):
    try:
        target_file_name = file_path.split("/")[-1]
        for file in files:
            current_file_name = file["filename"].split("/")[-1]
            if current_file_name == target_file_name:
                return file["raw_url"]
    except Exception as e:
        print(e)
        return None

In [8]:

for pull in pulls:
    review_comments = pull.get_review_comments()
    for review_comment in review_comments:
        pr_review_id = extract_data(review_comment.raw_data, "pull_request_review_id")

        commit_before_id = extract_data(pull.raw_data["base"], "sha")
        commit_while_id = extract_data(review_comment.raw_data, "original_commit_id")
        commit_after_id = extract_data(review_comment.raw_data, "commit_id")
        
        commit_before = get_commit(repo,commit_before_id)
        commit_while = get_commit(repo,commit_while_id)
        commit_after = get_commit(repo,commit_after_id)
       #print(commit_before.raw_data["files"][0])
        comment_data = {
            "comment_id" : extract_data(review_comment.raw_data, "id"),
            "commit_id" : extract_data(review_comment.raw_data, "commit_id"),
            "original_commit_id" : extract_data(review_comment.raw_data, "original_commit_id"),
            "user_id" : extract_data(review_comment.raw_data["user"], "id"),
            "comment" :  extract_data(review_comment.raw_data, "body"),
            "created_at" : extract_data(review_comment.raw_data, "created_at"),
            "updated_at" : extract_data(review_comment.raw_data, "updated_at"),
            "start_line" : extract_data(review_comment.raw_data, "start_line"),
            "original_start_line" : extract_data(review_comment.raw_data, "original_start_line"),
            "original_line" : extract_data(review_comment.raw_data, "original_line"),
            "side" : extract_data(review_comment.raw_data, "side"),
            "filename" : extract_data(review_comment.raw_data, "path"),
            "commit_before" : commit_before_id,
            "commit_while" : commit_while_id,
            "commit_after" : commit_after_id,
            "url_source_before" : extract_data(commit_before.raw_data["files"][0], "raw_url") if commit_before is not None else None,
            "url_source_while" : extract_data(commit_while.raw_data["files"][0], "raw_url") if commit_while is not None else None,
            "url_source_after" : extract_data(commit_after.raw_data["files"][0], "raw_url") if commit_after is not None else None,
            "file-content_before" : get_file_content(find_file(commit_before.raw_data["files"], review_comment.raw_data["path"])) if commit_before is not None else None,
            "file-content_while" : get_file_content(find_file(commit_while.raw_data["files"], review_comment.raw_data["path"])) if commit_while is not None else None,
            "file-content_after" : get_file_content(find_file(commit_after.raw_data["files"], review_comment.raw_data["path"])) if commit_after is not None else None,

        }

        if pr_review_id is not None:
            # in the review_datas find the review_id and add the comment to the review
            for current_review_data in review_datas:
                if current_review_data["review_id"] == pr_review_id:
                    current_review_data["comments"].append(comment_data)
                    break



    


In [9]:
review_datas

[{'project': 'apache/openmeetings',
  'pull_number': 103,
  'review_id': 542920858,
  'user': 'solomax',
  'user_id': 3870591,
  'message': 'What is the purpose to send messages to `Dweet`?\r\nI mean why would OM users would like to have this new feature?',
  'state': 'COMMENTED',
  'submitted_at': '2020-12-02T15:14:26Z',
  'pull_request_id': '103',
  'pull_request_url': 'https://api.github.com/repos/apache/openmeetings/pulls/103',
  'commit_id': '6971422b82b80aecdd9c5a73725c26a74efe11c0',
  'comments': [{'comment_id': 534233249,
    'commit_id': 'fd14c0e98e4d3da595828d74c3f07cddef613282',
    'original_commit_id': '6971422b82b80aecdd9c5a73725c26a74efe11c0',
    'user_id': 3870591,
    'comment': 'please do not create trailing spaces',
    'created_at': '2020-12-02T14:58:34Z',
    'updated_at': '2020-12-07T17:42:28Z',
    'start_line': None,
    'original_start_line': None,
    'original_line': 166,
    'side': 'RIGHT',
    'filename': 'openmeetings-web/pom.xml',
    'commit_before': '